_First code cell from custom template `dsml.ipynb` in [user_path]\Anaconda3\envs\dsml\Lib\site-packages\jupyterlab_templates\templates\jupyterlab_templates_. <br>
_See Tim Paine's [`jupyter_lab templates` extention](https://github.com/timkpaine/jupyterlab_templates)._

In [ ]:
import sys
from pathlib import Path, PurePath as PPath

print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = str(PPath(this_path).parent)
    else:
        newp = str(PPath(this_path)) 
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
from collections import defaultdict, OrderedDict
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp

# Filtered dir() for method discovery:
def filter_dir(obj, start_with_str='_', exclude=True):
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]

def get_mdl_pkgs(alib):
    import inspect
    "Inspect module hierarchy on two levels ony."
    for name, mdl in inspect.getmembers(alib, inspect.ismodule):
        print('\n{:>13} : {}'.format(mdl.__name__, filter_dir(mdl)))
        for mdl_name, mdl_sub in inspect.getmembers(mdl, inspect.ismodule):
            if mdl_sub.__doc__:
                print('\n{:>20} : {}'.format(mdl_name, mdl_sub.__doc__.strip()))
                

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import get_ipython
from IPython.display import HTML, Markdown, display #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    return HTML('<div style="{}">{}</div>'.format(style, title))


def add_div(div_class, div_start, div_text, output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output string, but the cell mode is still in 'code' not 'markdown':
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    The only thing to do is change the cell mode to Markdown.
    If `output_string=False`, the HTML output is displayed in an output cell.
    """
    accepted = ['info', 'warning', 'danger']
    if div_class not in accepted:
        return HTML(f"""<div class="alert"><b>Wrong class:</b> `div_start` is one of {accepted}.
                    </div>""")
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}</b><br>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div) #HTML(div)
    
    
def add_div_around_html(div_html_text, output_string=True, div_style="{width: 80%}"):
    """
    Wrap an html code str inside a div.
    div_style: whatever follows style= within the <div>
    
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output string (but the cell mode is still in 'code' not 'markdown')
    The only thing to do is change the cell mode to Markdown.
    If `output_string=False`, the HTML output is displayed in an output cell.
    """
    div = f"""<div style={div_style}>{div_html_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return HTML(div)
    
# autoreload extension
from IPython import get_ipython
ipython = get_ipython()

if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Report preparation - Udacity AI Project 2 Classical Planning Agent

* Process raw data obtained from modified `run_search.py` (`run_report.py`) into pandas DataFrames
* Create charts and analyses as per requirements
* Produce a html report printed as pdf

---
### The complete specifications of the problem can be found [in the Udacity AIND repo](https://github.com/udacity/artificial-intelligence/tree/master/Projects/2_Classical%20Planning). 

- [x] 1. Run example: `>python example_have_cake.py` in workspace
- [x] 2. Complete the TODO sections in `my_planning_graph.py` and implement the following functions (methods):  
 - [x] 2.1 `ActionLayer._inconsistent_effects`
 - [x] 2.2 `ActionLayer._interference`
 - [x] 2.3 `ActionLayer._competing_needs`
 - [x] 2.4 `LiteralLayer._inconsistent_support`
 - [x] 2.6 `LiteralLayer._negation`
 - [x] 2.7 `PlanningGraph.h_levelsum`
 - [x] 2.7 `PlanningGraph.h_maxlevel`  
 - [x] 2.7 `PlanningGraph.h_setlevel`  
 - [x] 2.8  Test solution by running `python -m unittest -v`

- [x] 3. Experiment with different search algorithms using `run_search.py`. 
  > The goal of your experiment is to understand the tradeoffs in speed, optimality, and complexity of progression search as problem size increases. 
 You can also run specific problems & search algorithms - e.g., to run breadth first search and UCS on problems 1 and 2: `python run_search.py -p 1 2 -s 1 2`

- [x] 4. Experiment with the planning algorithms
  > The run_search.py script allows you to choose any combination of 11 search algorithms (3 uninformed and 8 with heuristics) on 4 air cargo problems.

 - [x] 4.1 You should run all of the search algorithms on the first 2 problems and record the following information for each combination:
  - number of actions in the domain
  - number of new node expansions
  - time to complete the plan search  
  
- [x] 5. Use the results from the first 2 problems to determine whether _any of the **uninformed search** algorithms_ should be excluded for problems 3 and 4. 
  > You must run at least 1 uninformed search, 2 heuristics with greedy best first search, and 2 heuristics with A* on problems 3 and 4.

---
## Report Requirements

Your submission for review **must** include a report named "report.pdf" that includes all of the figures (charts or tables) and written responses to the questions below. You may plot multiple results for the same topic on the same chart or use multiple charts. (Hint: you may see more detail by using log space for one or more dimensions of these charts.)

- [x] Use a table or chart to analyze the number of nodes expanded against number of actions in the domain
- [x] Use a table or chart to analyze the search time against the number of actions in the domain
- [ ] Use a table or chart to analyze the length of the plans returned by each algorithm on all search problems

Use your results to answer these questions:
- Question 1: Which algorithm or algorithms would be most appropriate for planning in a very restricted domain (i.e., one that has only a few actions) and needs to operate in real time? 
- Question 2: Which algorithm or algorithms would be most appropriate for planning in very large domains (e.g., planning delivery routes for all UPS drivers in the U.S. on a given day)
- Question 3: Which algorithm or algorithms would be most appropriate for planning problems where it is important to find only optimal plans?

---
**Report evaluation**:

<div  style = "width:80%">
<table class="table table-bordered section-table"> <thead> <tr> <!-- ngIf: !ctrl.hideCriteria --><th class="rubric-category criteria col-xs-3 ng-scope" ng-if="!ctrl.hideCriteria"> <span translate="" class="ng-scope">Criteria</span> </th><!-- end ngIf: !ctrl.hideCriteria --> <th class="rubric-category meets-specs" ng-class="ctrl.reviewerTips ? col-xs-7 : col-xs-4"> <span translate="" class="ng-scope">Meets Specifications</span> </th> <!-- ngIf: ctrl.reviewerTips --> </tr> </thead> <tbody>  <!-- ngRepeat: rubricItem in section.rubric_items --><tr ng-repeat="rubricItem in section.rubric_items" class="ng-scope"> <!-- ngIf: !ctrl.hideCriteria --><td class="rubric-item col-xs-3 ng-binding ng-scope" ng-if="!ctrl.hideCriteria" ng-bind-html="localize(rubricItem, 'criteria', markup=true)"><p>Analyze the search complexity as a function of domain size, search algorithm, and heuristic.</p>
</td><!-- end ngIf: !ctrl.hideCriteria --> <td class="rubric-item ng-binding" ng-class="ctrl.reviewerTips ? col-xs-7 : col-xs-4" ng-bind-html="localize(rubricItem, 'passed_description', markup=true)"><p>Report includes a table or chart to analyze the number of nodes expanded against number of actions in the domain.</p>
<ul>
<li>The chart or table includes data for all search &amp; heuristic combinations for air cargo problems 1 and 2</li>
<li>The chart or table includes data <strong>at least</strong> one uninformed search, two heuristics with greedy best first search, and two heuristics with A* on air cargo problems 3 and 4</li>
<li>Report includes at least a one paragraph discussion of these results that analyzes the growth trends as the problem size increases</li>
</ul>
</td> <!-- ngIf: ctrl.reviewerTips --> </tr><!-- end ngRepeat: rubricItem in section.rubric_items --><tr ng-repeat="rubricItem in section.rubric_items" class="ng-scope"> <!-- ngIf: !ctrl.hideCriteria --><td class="rubric-item col-xs-3 ng-binding ng-scope" ng-if="!ctrl.hideCriteria" ng-bind-html="localize(rubricItem, 'criteria', markup=true)"><p>Analyze search time as a function of domain size, search algorithm, and heuristic.</p>
</td><!-- end ngIf: !ctrl.hideCriteria --> <td class="rubric-item ng-binding" ng-class="ctrl.reviewerTips ? col-xs-7 : col-xs-4" ng-bind-html="localize(rubricItem, 'passed_description', markup=true)"><p>Report includes a table or chart to analyze the search time against the number of actions in the domain.</p>
<ul>
<li>The chart or table includes data for all search &amp; heuristic combinations for air cargo problems 1 and 2</li>
<li>The chart or table includes data <strong>at least</strong> one uninformed search, two heuristics with greedy best first search, and two heuristics with A* on air cargo problems 3 and 4</li>
<li>Report includes at least a one paragraph discussion of these results that analyzes the growth trends as the problem size increases</li>
</ul>
</td> <!-- ngIf: ctrl.reviewerTips --> </tr><!-- end ngRepeat: rubricItem in section.rubric_items --><tr ng-repeat="rubricItem in section.rubric_items" class="ng-scope"> <!-- ngIf: !ctrl.hideCriteria --><td class="rubric-item col-xs-3 ng-binding ng-scope" ng-if="!ctrl.hideCriteria" ng-bind-html="localize(rubricItem, 'criteria', markup=true)"><p>Analyze the optimality of solution as a function of domain size, search algorithm, and heuristic.</p>
</td><!-- end ngIf: !ctrl.hideCriteria --> <td class="rubric-item ng-binding" ng-class="ctrl.reviewerTips ? col-xs-7 : col-xs-4" ng-bind-html="localize(rubricItem, 'passed_description', markup=true)"><p>Report includes a table or chart to analyze the length of the plans returned by each algorithm on all search problems.</p>
<ul>
<li>The chart or table includes data for all search &amp; heuristic combinations for air cargo problems 1 and 2</li>
<li>The chart or table includes data <strong>at least</strong> one uninformed search, two heuristics with greedy best first search, and two heuristics with A* on air cargo problems 3 and 4</li>
</ul>
</td> <!-- ngIf: ctrl.reviewerTips --> </tr><!-- end ngRepeat: rubricItem in section.rubric_items --><tr ng-repeat="rubricItem in section.rubric_items" class="ng-scope"> <!-- ngIf: !ctrl.hideCriteria --><td class="rubric-item col-xs-3 ng-binding ng-scope" ng-if="!ctrl.hideCriteria" ng-bind-html="localize(rubricItem, 'criteria', markup=true)"><p>Report answers all required questions</p>
</td><!-- end ngIf: !ctrl.hideCriteria --> <td class="rubric-item ng-binding" ng-class="ctrl.reviewerTips ? col-xs-7 : col-xs-4" ng-bind-html="localize(rubricItem, 'passed_description', markup=true)"><p>Submission includes a short answer to each of the following questions. (A short answer should be at least 1-2 sentences at most a small paragraph.)</p>
<ul>
<li>Which algorithm or algorithms would be most appropriate for planning in a very restricted domain (i.e., one that has only a few actions) and needs to operate in real time?</li>
<li>Which algorithm or algorithms would be most appropriate for planning in very large domains (e.g., planning delivery routes for all UPS drivers in the U.S. on a given day)</li>
<li>Which algorithm or algorithms would be most appropriate for planning problems where it is important to find only optimal plans?</li>
</ul>
</td> <!-- ngIf: ctrl.reviewerTips --> </tr><!-- end ngRepeat: rubricItem in section.rubric_items --> </tbody> </table>
</div>

---
# Report prep 

I modified `run_search.main()` and `_utils.run_search` to obtain a tab separated file to facilitate the processing of the results.
---

In [ ]:
# Create data subfolders for raw and processed files:
DATA_DIRS = []
for fldr in ['raw', 'processed']:
    p = DIR_DATA.joinpath(fldr)
    if not p.exists():
        Path.mkdir(p)
    DATA_DIRS.append(p)

### Initial results : all searches on problems 1 and 2

In [ ]:
import report as rpt
from display_figure import display_figure

In [ ]:
SEARCHES = rpt.SEARCHES
SEARCHES
    
# Problem names    
problems = rpt.problems

#Problem specifications (df)
specs = rpt.specs #rpt.get_prob_specs()
specs

---
## Data for p1:

In [ ]:
# File used if replace=False: DATA_DIRS[1].joinpath('prob_1_df.csv')

df1 = rpt.get_problem_data_df('prob_1', problems[0], DATA_DIRS[0], DATA_DIRS[1], file_as_tsv=True, replace=False)
df1.shape
df1

## Data for p2:  

In [ ]:
# File used if replace=False: DATA_DIRS[1].joinpath('prob_2_df.csv')

df2 = rpt.get_problem_data_df('prob_2', problems[1], DATA_DIRS[0], DATA_DIRS[1], file_as_tsv=True, replace=False)
df2.shape
df2

---
## Chart creation:

In [ ]:
fig1 = DIR_IMG.joinpath('p12_NewNodes_Actions.png')
fig2 = DIR_IMG.joinpath('p12_ElapsedSeconds_Actions.png')

done = True

if not done:
    # First plot: 'NewNodes' vs 'Actions'
    rpt.make_bar_plots([df1, df2],
                   'Actions', 'NewNodes',
                   [problems[0], problems[1]],
                   legend_bbox=(1.05, .95),
                   to_file=fig1)

    # Second plot: 'ElapsedSeconds' vs 'Actions'
    rpt.make_bar_plots([df1, df2],
                   'Actions', 'ElapsedSeconds',
                   [problems[0], problems[1]],
                   legend_bbox=(1.05, .95),
                   to_file=fig2)

# Analysis for Problems 1 & 2

In [ ]:
specs_ml = specs[specs.columns[1:]].to_html(index=False, justify='center')
replace_str1 = ' style="text-align: center;"'
replace_str2 = 'class="dataframe"'
specs_ml = specs_ml.replace(replace_str1, '')
specs_ml = specs_ml.replace(replace_str2, replace_str1)

analysis = f"""
    <h1> Analysis of the output of Problems 1 and 2</h1>
    <p style="font-size:110%;">
        All four problems differ by the number of nodes, cargo loads, airports, and specificity of goals. 
        <pre>
            {specs_ml}
        </pre>
    </p>
    <p style="font-size:110%;">
        The first two problems are the less complex and are used to decide on the appropriate choices of search 
        functions and heuristic on more "real life" problems (Problems 3 and 4).<br>
        The analysis relies on three charts:
        <ul>
            <li> Number of nodes expanded against number of actions in the domain</li>
            <li> Search time against the number of actions in the domain</li>
            <li> Length of the plans returned by each algorithm on all search problems</li>
        </ul>
    </p>
"""

In [ ]:
rpt.add_div_around_html(analysis, output_string=False)

## Chart display

In [ ]:
style_dict = {'div': {'width': 7},
              'figure': {'display':'inline-block', 'text-align':'left'},
              'image': {'display':'block', 'width': 600, 'height':500},
              'caption': {'color':'teal','font-weight':'bold', 'font-family': 'Arial, Helvetica, sans-serif'}
             }

caption1_dict = {'number': 1,
                'caption': 'NewNodes expanded in each search function for Problems 1 and 2.&nbsp;&nbsp;(Note the log scale for Problem 2.)'}

caption2_dict = {'number': 2,
                'caption': 'ElapsedSeconds in each search function for Problems 1 and 2.'}

In [ ]:
display_figure(fig1, style_dict, caption1_dict, img_title='NewNodes', return_html=True)
display_figure(fig2, style_dict, caption2_dict, img_title='Seconds', return_html=True)

In [ ]:
candidates = rpt.get_elim_candidates(df2, df1)
candidates

In [ ]:
insights = rpt.paragraph_p12(candidates, return_html=True)
#rpt.add_div_around_html(insights)

In [ ]:
rpt.add_div_around_html(insights, output_string=False) #output_string=True in final rpt

---
# Problems 3 & 4
## Intro, data

In [ ]:
p34_intro = """
<h2>Air cargo problems 3 and 4:</h2>
   <p style="font-size:110%;">
    After elimination of problems 3, 8, and 10, the searches performed on both Problems 3 & 4 were the following<br>
    (with the number corresponds to the number passed to `run_report.py` in the -s argument):
    </p>
    <pre> 
      <b>Uninformed searches </b>
        1: breadth_first_search
        2: depth_first_graph_search
      <b>Informed searches with two different heuristics </b>
        4: greedy_best_first_graph_search + h_unmet_goals
        5: greedy_best_first_graph_search + h_pg_levelsum
        6: greedy_best_first_graph_search + h_pg_maxlevel
        7: greedy_best_first_graph_search + h_pg_setlevel
        9: astar_search + h_pg_levelsum
        11: <mark>astar_search + h_pg_setlevel</mark>
    </pre>
    <p style="font-size:110%;">
      Note: Search <mark>11</mark> was aborted on both problems due to excessive run time (> 1 hour).<br>
      The raw data files were generated at the comman line using the `run_report.py` module, 
      which is a copy of the original `run_search.py` module with modifications on output string formats. 
      Note that `_utils.py.run_search` was amended accordingly.
    </p>
""" 
rpt.add_div_around_html(p34_intro, output_string=False)

In [ ]:
#p3_out = DIR_DATA.joinpath('prob_3_df.csv')

df3 = rpt.get_problem_data_df('prob_3', problems[2], DATA_DIRS[0], DATA_DIRS[1], file_as_tsv=True, replace=False)
df3.shape
df3

In [ ]:
#p4_out = DIR_DATA.joinpath('prob_4_df.csv')

df4 = rpt.get_problem_data_df('prob_4', problems[3], DATA_DIRS[0], DATA_DIRS[1], file_as_tsv=True, replace=False)
df4.shape
df4

---
# Problems 3 & 4
## Chart creation

In [ ]:
fig3 = DIR_IMG.joinpath('p34_NewNodes_Actions.png')
fig4 = DIR_IMG.joinpath('p34_ElapsedSeconds_Actions.png')

done = True

if not done:
    # First plot: 'NewNodes' vs 'Actions'
    rpt.make_bar_plots([df3, df4],
                       'Actions', 'NewNodes',
                       [problems[2], problems[3]],
                       legend_bbox=(1.05, .95),
                       to_file=fig3, 
                       excluded=candidates)

    # Second plot: 'ElapsedSeconds' vs 'Actions'
    rpt.make_bar_plots([df3, df4],
                       'Actions', 'ElapsedSeconds',
                       [problems[2], problems[3]],
                       legend_bbox=(1.05, .95),
                       to_file=fig4,
                       excluded=candidates)

## Chart display:

In [ ]:
style_dict = {'div': {'width': 7},
              'figure': {'display':'inline-block', 'text-align':'left'},
              'image': {'display':'block', 'width': 600, 'height':500},
              'caption': {'color':'teal','font-weight':'bold', 'font-family': 'Arial, Helvetica, sans-serif'}
             }

cap3 = """NewNodes expanded in each search function for Problems 3 and 4.<br>
<pre>         <mark>Search 11, A* with setlevel heuristic</mark>: aborted after 1 hour.<pre>"""
caption3_dict = {'number': 3, 'caption': cap3}

cap4 = """Elapsed minutes in each search function for Problems 1 and 2.<br>
<pre>         <mark>Search 11, A* with setlevel heuristic</mark>: aborted after 1 hour.<pre>"""
caption4_dict = {'number': 4, 'caption': cap4}


display_figure(fig3, style_dict, caption3_dict, img_title='NewNodes')
display_figure(fig4, style_dict, caption4_dict, img_title='Minutes')

---
# All problems: PlanLength analysis

* Use a table or chart to analyze the length of the plans returned by each algorithm on all search problems

In [ ]:
dfa = rpt.concat_all_dfs([df1, df2, df3, df4])
dfa_rows = dfa.shape[0]
dfa.head()

In [ ]:
dfa = rpt.concat_all_dfs([df1, df2, df3, df4])

dbl_tbl, dbl_para, df_dbl = rpt.plans_length(dfa, 'double')
sgl_tbl, sgl_para, df_sgl = rpt.plans_length(dfa, 'single')

rpt.add_div_around_html(dbl_para, output_string=False)
rpt.add_div_around_html(sgl_para, output_string=False)

tbls_html = """
<div style="float:left; width:100%; padding:1px;">
  <div style="float:left; width:50%; padding:2px;">
    {}
   </div>
  <div style="float:left; width:50%; padding:2px;">
    {}
  </div>
</div>
"""
tables_in_row = tbls_html.format(dbl_tbl, sgl_tbl)

#rpt.add_div_around_html(tables_in_row, output_string=true)

---
# Answers

In [ ]:
# A1
A1 = dfa[(dfa.ElapsedSeconds) < 1 & (dfa.Actions==20)].sort_values('ElapsedSeconds', ascending=True)
A1_searchers = sorted(A1.id.unique())
A1_searchers

In [ ]:
# A2
A2 = dfa[(dfa.ElapsedSeconds) < 1 & (dfa.Actions > 100)].sort_values('ElapsedSeconds', ascending=True)
A2_searchers = sorted(A2.id.unique())
A2_searchers

<h3>* Anwers to questions</h3>
<blockquote class="w3-panel w3-leftbar w3-light-grey">
<strong>Question 1:</strong>
<p style="font-size:110%;">
Which algorithm(s) would be most appropriate for planning in a very restricted domain
(i.e., one that has only a few actions) and needs to operate in real time?
</p>
</blockquote>
<blockquote class="w3-panel w3-leftbar w3-light-grey">
<strong>&#187; Answer:</strong>
<p style="font-size:110%;">
A domain with few actions would be like Problem 1. All search algorithms would be appropriate in this case as they all performed under 1 second.
</p>
</blockquote>

<blockquote class="w3-panel w3-leftbar w3-light-grey">
<strong>Question 2:</strong>
<p style="font-size:110%;">
Which algorithm or algorithms would be most appropriate for planning in very large domains 
(e.g., planning delivery routes for all UPS drivers in the U.S. on a given day.)
</p>
</blockquote>
<blockquote class="w3-panel w3-leftbar w3-light-grey">
<strong>&#187; Answer:</strong>
<p style="font-size:110%;">
For problems with more than 100 and with a search time restricted to under 1 second, algorithm <b>4</b>: <i>greedy_best_first_graph_search h_unmet_goals</i> would be the best.
</p>
</blockquote>

<blockquote class="w3-panel w3-leftbar w3-light-grey">
<strong>Question 3:</strong>
<p style="font-size:110%;">
Which algorithm(s) would be most appropriate for planning problems where it is important to find only optimal plans?
</p>
</blockquote>
<blockquote class="w3-panel w3-leftbar w3-light-grey">
<strong>&#187; Answer:</strong>
<p style="font-size:110%;">
The algorithms that guarantee to find the shortest path and are thus optimal are: <b>1</b>: <i>breadth_first_search</i> and <b>3</b>: <i>uniform_cost_search</i>.
</p>
</blockquote>


---
---

---
# Plan length analysis :  "manual code"

In [ ]:
# sgle_digit_plans = dfa[dfa.PlanLength <= 9].sort_values(['PlanLength'], ascending=False) before function

sgle_digit_plans = df_sgl
sgle_uniq_probs =sgle_digit_plans['Air cargo problem'].unique()
sd_plans = sgle_digit_plans.shape[0]
sd_searcher_cnt = sgle_digit_plans['Searcher'].value_counts()
sd_fn_cnt = sgle_digit_plans['search_fn'].value_counts()

sgle_digit_plans
sgle_digit_plans.sort_values('ElapsedSeconds', ascending=True).head()

sf_fn = sd_fn_cnt.to_frame()
sf_fn.reset_index(drop=False, inplace=True)
sf_fn.columns = ['Search function','Frequency where PlanLength<10'] 
sf_fn_ml = sf_fn.to_html(index=False, justify='center')

replace_str1 = ' style="text-align: center;"'
replace_str2 = 'class="dataframe"'
sf_fn_ml = sf_fn_ml.replace(replace_str1, '')
sf_fn_ml = sf_fn_ml.replace(replace_str2, replace_str1)
rpt.add_div_around_html(sf_fn_ml, output_string=False)

pct_sd = sd_plans/dfa_rows
top2_fn = sd_fn_cnt[0:2].sum()
pct_top2_fn = top2_fn/sd_plans

text = f"Out of {dfa_rows} completed searches, {pct_sd:.0%} ({sd_plans}), have single-digit PlanLength.<br>"
text += f"In that subset, {top2_fn:d} ({pct_top2_fn:.0%}) involve the search functions `{sd_fn_cnt.index[0]}` and `{sd_fn_cnt.index[1]}`."
 
if len(sgle_uniq_probs) < 4:
    text += " And this occurs only for Problems: "
    pro = ",".join('{}' for p in sgle_uniq_probs) +'.<br>'
    text += pro.format(*sgle_uniq_probs)
else:
    text += " And this occurs for all Problems."
    
Markdown(text)

---